<a href="https://colab.research.google.com/github/danieldrako/Algoritmos-Clasificacion-de-Texto/blob/main/02AlgoritmoHMM_Viterbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalacion de dependencias previas
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'UD_Spanish-AnCora'...
remote: Enumerating objects: 928, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 928 (delta 128), reused 174 (delta 121), pack-reused 747
Receiving objects: 100% (928/928), 337.55 MiB | 35.60 MiB/s, done.
Resolving deltas: 100% (653/653), done.


# Carga del modelo HMM previamente entrenado

In [2]:
# cargamos las probabilidades del modelo HMM
import numpy as np
transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()
emissionProbdict = np.load('emissionHMM.npy', allow_pickle='TRUE').item()

In [3]:
# identificamos las categorias gramaticales 'upos' unicas en el corpus
stateSet = set([w.split('|')[1] for w in list(emissionProbdict.keys())])
stateSet

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SYM',
 'VERB',
 '_'}

In [4]:
# enumeramos las categorias con numeros para asignar a 
# las columnas de la matriz de Viterbi
tagStateDict = {}
for i, state in enumerate(stateSet):
  tagStateDict[state] = i
tagStateDict

{'NOUN': 0,
 'CCONJ': 1,
 'PROPN': 2,
 'AUX': 3,
 'ADV': 4,
 'NUM': 5,
 '_': 6,
 'DET': 7,
 'SCONJ': 8,
 'VERB': 9,
 'ADJ': 10,
 'INTJ': 11,
 'PART': 12,
 'PRON': 13,
 'SYM': 14,
 'PUNCT': 15,
 'ADP': 16}

# Distribucion inicial de estados latentes

In [5]:
# Calculamos distribución inicial de estados
initTagStateProb = {} # \rho_i^{(0)}
from conllu import parse_incr 
wordList = []
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")
count = 0 # cuenta la longitud del corpus
for tokenlist in parse_incr(data_file):
  count += 1
  tag = tokenlist[0]['upos']
  if tag in initTagStateProb.keys():
    initTagStateProb[tag] += 1
  else:
    initTagStateProb[tag] = 1

for key in initTagStateProb.keys():
  initTagStateProb[key] /= count

initTagStateProb

{'DET': 0.36275695284159615,
 'PROPN': 0.1124546553808948,
 'ADP': 0.15538089480048367,
 'PRON': 0.06348246674727932,
 'SCONJ': 0.02418379685610641,
 'ADV': 0.056831922611850064,
 'PUNCT': 0.08222490931076179,
 'VERB': 0.021160822249093107,
 'ADJ': 0.010882708585247884,
 'CCONJ': 0.032648125755743655,
 'NOUN': 0.02720677146311971,
 '_': 0.009068923821039904,
 'INTJ': 0.0006045949214026602,
 'AUX': 0.019347037484885126,
 'NUM': 0.01995163240628779,
 'PART': 0.0018137847642079807}

In [6]:
# verificamos que la suma de las probabilidades es 1 (100%)
#np.array([initTagStateProb[k] for k in initTagStateProb.keys()]).sum() 
#O de otro modo
np.array(list(initTagStateProb.values())).sum() 

1.0

# Construcción del algoritmo de Viterbi






Dada una secuencia de palabras $\{p_1, p_2, \dots, p_n \}$, y un conjunto de categorias gramaticales dadas por la convención `upos`, se considera la matriz de probabilidades de Viterbi así:

$$
\begin{array}{c c}
\begin{array}{c c c c}
\text{ADJ} \\
\text{ADV}\\
\text{PRON} \\
\vdots \\
{}
\end{array} 
&
\left[
\begin{array}{c c c c}
\nu_1(\text{ADJ}) & \nu_2(\text{ADJ}) & \dots  & \nu_n(\text{ADJ})\\
\nu_1(\text{ADV}) & \nu_2(\text{ADV}) & \dots  & \nu_n(\text{ADV})\\ 
\nu_1(\text{PRON}) & \nu_2(\text{PRON}) & \dots  & \nu_n(\text{PRON})\\
\vdots & \vdots & \dots & \vdots \\ \hdashline
p_1 & p_2 & \dots & p_n 
\end{array}
\right] 
\end{array}
$$

Donde las probabilidades de la primera columna (para una categoria $i$) están dadas por: 

$$
\nu_1(i) = \underbrace{\rho_i^{(0)}}_{\text{probabilidad inicial}} \times \underbrace{P(p_1 \vert i)}_{\text{emisión}}
$$

luego, para la segunda columna (dada una categoria $j$) serán: 

$$
\nu_2(j) = \max_i \{ \nu_1(i) \times \underbrace{P(j \vert i)}_{\text{transición}} \times \underbrace{P(p_2 \vert j)}_{\text{emisión}} \}
$$

así, en general las probabilidades para la columna $t$ estarán dadas por: 

$$
\nu_{t}(j) = \max_i \{ \overbrace{\nu_{t-1}(i)}^{\text{estado anterior}} \times \underbrace{P(j \vert i)}_{\text{transición}} \times \underbrace{P(p_t \vert j)}_{\text{emisión}} \}
$$

In [7]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize # importamos el tokenizador

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [24]:
#@title Función que retorna la matriz de viterbi que corresponde a la secuencia de palbras que pasarán como argumento.
# tambien necesita las matrices de probabilidad, el diccionario de categorias y el diccionario que relaciona las categorias con sus filas en la matriz
def ViterbiMatrix(secuencia, transitionProbdict=transitionProbdict, emissionProbdict=emissionProbdict, 
            tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
  seq = word_tokenize(secuencia)#tokenizador
  viterbiProb = np.zeros((17, len(seq)))  # upos tiene 17 categorias, matriz llena de ceros

  # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = seq[0].lower()+'|'+key
    if word_tag in emissionProbdict.keys():# si esta pareja etiqueta|palabra está en el diccionario de probabilidades de emisión
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbdict[word_tag] #tokenizador

  # computo de las siguientes columnas
  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbdict.keys():
        # miramos estados de la col anterior
        possible_probs = []
        for key2 in tagStateDict.keys(): 
          tag_row2 = tagStateDict[key2]
          tag_prevtag = key+'|'+key2
          if tag_prevtag in transitionProbdict.keys():
            if viterbiProb[tag_row2, col-1]>0:
              possible_probs.append(
                  viterbiProb[tag_row2, col-1]*transitionProbdict[tag_prevtag]*emissionProbdict[word_tag])
        #viterbiProb[tag_row, col] = max(possible_probs)
  
  return viterbiProb

matrix = ViterbiMatrix('La palabra es muy grande')
matrix

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.08447641, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.00057423, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.

In [25]:
#@title Función que retorna los tags
def ViterbiTags(secuencia, transitionProbdict=transitionProbdict, emissionProbdict=emissionProbdict, 
            tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
  seq = word_tokenize(secuencia)
  viterbiProb = np.zeros((17, len(seq)))  # upos tiene 17 categorias

  # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = seq[0].lower()+'|'+key
    if word_tag in emissionProbdict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbdict[word_tag]

  # computo de las siguientes columnas
  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbdict.keys():
        # miramos estados de la col anterior
        possible_probs = []
        for key2 in tagStateDict.keys(): 
          tag_row2 = tagStateDict[key2]
          tag_prevtag = key+'|'+key2
          if tag_prevtag in transitionProbdict.keys():
            if viterbiProb[tag_row2, col-1]>0:
              possible_probs.append(
                  viterbiProb[tag_row2, col-1]*transitionProbdict[tag_prevtag]*emissionProbdict[word_tag])
        #viterbiProb[tag_row, col] = max(possible_probs)

    # contruccion de secuencia de tags
    res = []
    for i, p in enumerate(seq):
      for tag in tagStateDict.keys():
        if tagStateDict[tag] == np.argmax(viterbiProb[:, i]): #cada palabra está asociada a una columna
          res.append((p, tag))
      
  return res

matrix = ViterbiMatrix('La palabra es muy grande')

In [26]:
ViterbiTags('La palabra es muy grande')

[('La', 'DET'),
 ('palabra', 'NOUN'),
 ('es', 'NOUN'),
 ('muy', 'NOUN'),
 ('grande', 'NOUN')]

# Entrenamiento directo de HMM con NLTK

* clase en python (NLTK) de HMM: https://www.nltk.org/_modules/nltk/tag/hmm.html

In [28]:
#@title ejemplo con el Corpus Treebank en ingles
import nltk
nltk.download('treebank')
from nltk.corpus import treebank
train_data = treebank.tagged_sents()[:3900]#frases tageadas hasta la sentencia 3900

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [29]:
#@title estructura de la data de entrenamiento
train_data

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]

In [30]:
#@title HMM pre-construido en NLTK
from nltk.tag import hmm
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(train_data) #esta parte hace el entraniento supervisado  del modelo de cadenas de Markov sobre el train_data
tagger

<HiddenMarkovModelTagger 46 states and 12385 output symbols>

In [34]:
tagger.tag("Dwayne The Rock Johnson will get old".split())

[('Dwayne', 'NNP'),
 ('The', 'NNP'),
 ('Rock', 'NNP'),
 ('Johnson', 'NNP'),
 ('will', 'NNP'),
 ('get', 'NNP'),
 ('old', 'NNP')]

In [35]:
tagger.tag("David will get old".split())

[('David', 'NNP'), ('will', 'MD'), ('get', 'VB'), ('old', 'JJ')]

In [36]:
#@title training accuracy
tagger.evaluate(treebank.tagged_sents()[:3900])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


0.9815403947224078

## Ejercicio de práctica

**Objetivo:** Entrena un HMM usando la clase `hmm.HiddenMarkovModelTrainer()` sobre el dataset `UD_Spanish_AnCora`.

1. **Pre-procesamiento:** En el ejemplo anterior usamos el dataset en ingles `treebank`, el cual viene con una estructura diferente a la de `AnCora`, en esta parte escribe código para transformar la estructura de `AnCora` de manera que quede igual al `treebank` que usamos así:

$$\left[ \left[ (\text{'El'}, \text{'DET'}), (\dots), \dots\right], \left[\dots \right] \right]$$

In [38]:
# desarrolla tu código aquí 
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'UD_Spanish-AnCora' already exists and is not an empty directory.


In [39]:
#leyendo el corpus AnCora
from conllu import parse_incr 
data_file = open("UD_Spanish-AnCora/es_ancora-ud-train.conllu", "r", encoding="utf-8")
data_array = []
for tokenlist in parse_incr(data_file):
  tokenized_text = []
  for token in tokenlist:
    tokenized_text.append((token['form'], token['upos']))
  data_array.append(tokenized_text)




In [44]:
data_array[:2]

[[('Las', 'DET'),
  ('reservas', 'NOUN'),
  ('de', 'ADP'),
  ('oro', 'NOUN'),
  ('y', 'CCONJ'),
  ('divisas', 'NOUN'),
  ('de', 'ADP'),
  ('Rusia', 'PROPN'),
  ('subieron', 'VERB'),
  ('800', 'NUM'),
  ('millones', 'NOUN'),
  ('de', 'ADP'),
  ('dólares', 'NOUN'),
  ('y', 'CCONJ'),
  ('el', 'DET'),
  ('26', 'NUM'),
  ('de', 'ADP'),
  ('mayo', 'NOUN'),
  ('equivalían', 'VERB'),
  ('a', 'ADP'),
  ('19.100', 'NUM'),
  ('millones', 'NOUN'),
  ('de', 'ADP'),
  ('dólares', 'NOUN'),
  (',', 'PUNCT'),
  ('informó', 'VERB'),
  ('hoy', 'ADV'),
  ('un', 'DET'),
  ('comunicado', 'NOUN'),
  ('del', '_'),
  ('de', 'ADP'),
  ('el', 'DET'),
  ('Banco', 'PROPN'),
  ('Central', 'PROPN'),
  ('.', 'PUNCT')],
 [('Según', 'ADP'),
  ('el', 'DET'),
  ('informe', 'NOUN'),
  (',', 'PUNCT'),
  ('el', 'DET'),
  ('19', 'NUM'),
  ('de', 'ADP'),
  ('mayo', 'NOUN'),
  ('las', 'DET'),
  ('reservas', 'NOUN'),
  ('de', 'ADP'),
  ('oro', 'NOUN'),
  ('y', 'CCONJ'),
  ('divisas', 'NOUN'),
  ('del', '_'),
  ('de', 'ADP'),
  

2. **Entrenamiento:** Una vez que el dataset esta con la estructura correcta, utiliza la clase `hmm.HiddenMarkovModelTrainer()` para entrenar con el $80 \%$ del dataset como conjunto de `entrenamiento` y $20 \%$ para el conjunto de `test`.

**Ayuda:** Para la separacion entre conjuntos de entrenamiento y test, puedes usar la funcion de Scikit Learn: 

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

En este punto el curso de Machine Learning con Scikit Learn es un buen complemento para entender mejor las funcionalidades de Scikit Learn: https://platzi.com/cursos/scikitlearn-ml/ 

In [45]:
# desarrolla tu código aquí

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_array, test_size=0.2, random_state=42)
print(len(train_data))
print(len(test_data))


11429
2858


In [46]:
from nltk.tag import hmm
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(train_data)
tagger

<HiddenMarkovModelTagger 18 states and 34366 output symbols>

3. **Validación del modelo:** Un vez entrenado el `tagger`, calcula el rendimiento del modelo (usando `tagger.evaluate()`) para los conjuntos de `entrenamiento` y `test`.



In [47]:
#desarrolla tu código aquí
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(data_array)

In [48]:
data_file = open("UD_Spanish-AnCora/es_ancora-ud-train.conllu", "r", encoding="utf-8")
test_array = []
for tokenlist in parse_incr(data_file):
  tokenized_text = []
  for token in tokenlist:
    tokenized_text.append((token['form'], token['upos']))
  test_array.append(tokenized_text)
len(test_array)

14287

In [50]:
# Instalamos conllu para leer el corpus
!pip install conllu
from conllu import parse_incr 
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") 

# Bajamos el corpus de AnCora
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

# Hacemos la transformacion del corpus al formato requerido
wordList = []
for tokenlist in parse_incr(data_file): 
  wordList2 = []
  for token in tokenlist:
    tag = token['upos']
    valor = token['form']
    wordList2.append((valor,tag)) 
  wordList.append(wordList2)

import nltk
from nltk.tag import hmm
from sklearn.model_selection import train_test_split

# Separamos el corpus
wordList_train, wordList_test= train_test_split(wordList, test_size=0.20, random_state=42)

# Entrenamos el modelo
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(wordList_train)
tagger

print(tagger.evaluate(wordList_test))
print(tagger.evaluate(wordList_train))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'UD_Spanish-AnCora' already exists and is not an empty directory.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.


0.2773140078891845


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.


0.9858684192817568


In [ ]:
#Se Podria decir que el modelo está sobre ajustado 